# Many redundant libraries are loaded, some have different names so need to agree on one and update uses throughout the code

In [ ]:
#importing all necessary libraries 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import time
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from tqdm import tnrange, tqdm_notebook
from sklearn.metrics import (confusion_matrix, 
                           accuracy_score)
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import datasets, metrics, model_selection, svm
from sklearn.metrics import precision_recall_curve

## Data quality check / cleaning / preparation 

Put code with comments. The comments should explain the code such that it can be easily understood. You may put text *(in a markdown cell)* before a large chunk of code to explain the overall purpose of the code, if it is not intuitive. **Put the name of the person / persons who contributed to each code chunk / set of code chunks.** An example is given below.

### Data quality check
*By Elton John*

The code below visualizes the distribution of all the variables in the dataset, and their association with the response.

In [7]:
#...Distribution of continuous variables...#

In [8]:
#...Distribution of categorical variables...#

In [9]:
#...Association of the response with the predictors...#

### Data cleaning
*By Xena Valenzuela*

From the data quality check we realized that:

1. Some of the columns that should have contained only numeric values, specifically <>, <>, and <> have special characters such as \*, #, %. We'll remove these characters, and convert the datatype of these columns to numeric.

2. Some of the columns have more than 60% missing values, and it is very difficult to impute their values, as the values seem to be missing at random with negligible association with other predictors. We'll remove such columns from the data.

3. The column `number_of_bedrooms` has some unreasonably high values such as 15. As our data consist of single-family homes in Evanston, we suspect that any value greater than 5 may be incorrect. We'll replace all values that are greater than 5 with an estimate obtained using the $K$-nearest neighbor approach.

4. The columns `house_price` has some unreasonably high values. We'll tag all values greater than 1 billion dollars as "potentially incorrect observation", to see if they distort our prediction / inference later on.

The code below implements the above cleaning.

In [ ]:
#...Code with comments...#

### Data preparation
*By Sankaranarayanan Balasubramanian and Chun-Li*

The following data preparation steps helped us to prepare our data for implementing various modeling / validation techniques:

1. Since we need to predict house price, we derived some new predictors *(from existing predictors)* that intuitively seem to be helpuful to predict house price. 

2. We have shuffled the dataset to prepare it for K-fold cross validation.

3. We have created a standardized version of the dataset, as we will use it to develop Lasso / Ridge regression models.

In [3]:
######---------------Creating new predictors----------------#########

#Creating number of bedrooms per unit floor area

#Creating ratio of bathrooms to bedrooms

#Creating ratio of carpet area to floor area

In [ ]:
######-----------Shuffling the dataset for K-fold------------#########

In [ ]:
######-----Standardizing the dataset for Lasso / Ridge-------#########

## Exploratory data analysis

Put code with comments. The comments should explain the code such that it can be easily understood. You may put text *(in a markdown cell)* before a large chunk of code to explain the overall purpose of the code, if it is not intuitive. **Put the name of the person / persons who contributed to each code chunk / set of code chunks.**

## Developing the model

**All of the folllowing code produced by Hiba Khatib & Emily Leibfritz**

Put code with comments. The comments should explain the code such that it can be easily understood. You may put text *(in a markdown cell)* before a large chunk of code to explain the overall purpose of the code, if it is not intuitive. **Put the name of the person / persons who contributed to each code chunk / set of code chunks.**

In [ ]:
final_train = train_with_age.drop(columns = ['PassengerId', 'Name', 'Ticket'])

In [ ]:
logit_model1 = sm.logit(formula = 'Survived~ Pclass + Sex + Age + SibSp + Parch + Fare + Embarked + age_binned', data = final_train).fit()
logit_model1.summary()

In [ ]:
final_train = train_with_age.drop(columns = ['PassengerId', 'Name', 'Ticket'])

In [ ]:
final_train = pd.get_dummies(final_train, columns = ['Sex'], drop_first = True)

In [ ]:
final_train.rename(columns={"Sex_male": "Sex"}, inplace = True)

In [ ]:
final_train['Embarked'] = final_train['Embarked'].apply(lambda x: 1 if x == 'S' else 
                                                               2 if x == 'C' else 
                                                               0)

In [ ]:
X_train = final_train[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]

In [ ]:
y = final_train.Survived
X = final_train.drop('Survived', axis = 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 45)

In [ ]:
X_train = X_train.drop('age_binned', axis = 1)
X_test = X_test.drop('age_binned', axis = 1)

In [ ]:
sklearn_model = LogisticRegression(max_iter=500)
sklearn_model.fit(X_train, y_train)

In [ ]:
y_pred_test = sklearn_model.predict(X_test)
print('Accuracy of logistic regression on test set : {:.4f}'.format(sklearn_model.score(X_test, y_test)))

In [ ]:
def fit_logistic_reg(X,Y):
    #Fit linear regression model and return RSS and R squared values
    model_k = linear_model.LogisticRegression()
    model_k.fit(X,Y)
    RSS = mean_squared_error(Y,model_k.predict(X)) * len(Y)
    R_squared = model_k.score(X,Y)
    return RSS, R_squared

In [ ]:
k = 11
RSS_list, R_squared_list, feature_list = [],[], []
numb_features = []

#Looping over k = 1 to k = 11 features in X
for k in tnrange(1,len(X_train.columns) + 1, desc = 'Loop...'):

    #Looping over all possible combinations: from 11 choose k
    for combo in itertools.combinations(X_train.columns,k):
        tmp_result = fit_logistic_reg(X_train[list(combo)],y_train)   #Store temp result 
        RSS_list.append(tmp_result[0])                  #Append lists
        R_squared_list.append(tmp_result[1])
        feature_list.append(combo)
        numb_features.append(len(combo))   

#Store in DataFrame
df = pd.DataFrame({'numb_features': numb_features,'RSS': RSS_list, 'R_squared':R_squared_list,'features':feature_list})

In [ ]:
df_min = df[df.groupby('numb_features')['RSS'].transform(min) == df['RSS']]
df_max = df[df.groupby('numb_features')['R_squared'].transform(max) == df['R_squared']]
display(df_min.head(10))
display(df_max.head(10))

In [ ]:
df['min_RSS'] = df.groupby('numb_features')['RSS'].transform(min)
df['max_R_squared'] = df.groupby('numb_features')['R_squared'].transform(max)
df.head()

In [ ]:
fig = plt.figure(figsize = (16,6))
ax = fig.add_subplot(1, 2, 1)

ax.scatter(df.numb_features,df.RSS, alpha = .2, color = 'darkblue' )
ax.set_xlabel('# Features')
ax.set_ylabel('RSS')
ax.set_title('RSS - Best subset selection')
ax.plot(df.numb_features,df.min_RSS,color = 'r', label = 'Best subset')
ax.legend()

ax = fig.add_subplot(1, 2, 2)
ax.scatter(df.numb_features,df.R_squared, alpha = .2, color = 'darkblue' )
ax.plot(df.numb_features,df.max_R_squared,color = 'r', label = 'Best subset')
ax.set_xlabel('# Features')
ax.set_ylabel('R squared')
ax.set_title('R_squared - Best subset selection')
ax.legend()

plt.show()

In [ ]:
df[(df["numb_features"] == 6)].sort_values(by = ['R_squared'], ascending=False)

In [ ]:
df[(df["numb_features"] == 7)].sort_values(by = ['R_squared'], ascending=False)

In [1]:
### Selected variables: (Pclass, Age, SibSp, Parch, Embarked, Sex)	

In [ ]:
X_train = X_train.drop('Fare', axis = 1)
X_test = X_test.drop('Fare', axis = 1)

In [ ]:
sklearn_model = LogisticRegression()
sklearn_model.fit(X_train, y_train)

y_pred_test = sklearn_model.predict(X_test)
print('Accuracy of logistic regression on test set : {:.4f}'.format(sklearn_model.score(X_test, y_test)))

In [ ]:
final_train.dtypes

In [ ]:
final_train['SibSp'] = pd.to_numeric(final_train['SibSp'])
final_train['Parch'] = pd.to_numeric(final_train['Parch'])
final_train['Pclass'] = pd.to_numeric(final_train['Pclass'])

In [ ]:
# Interacation between SibSp and Parch

def jitter(values,j):
    return values + np.random.normal(j,0.02,values.shape)
sns.scatterplot(x=jitter(final_train.SibSp,0), y = jitter(final_train.Parch,0),data = final_train, color = 'orange')

In [ ]:
# SibSP
sns.scatterplot(x = jitter(final_train.SibSp,0), y = jitter(final_train.Survived,0), color = 'orange')
model = smf.logit(formula = 'Survived~SibSp', data = final_train).fit()
sns.lineplot(x = 'SibSp', y= model.predict(final_train), data = final_train, color = 'blue') 
model.llf

In [ ]:
# SibSP squared
sns.scatterplot(x = jitter(final_train.SibSp,0), y = jitter(final_train.Survived,0), color = 'orange')
model = smf.logit(formula = 'Survived~SibSp+I(SibSp**2)', data = final_train).fit()
sns.lineplot(x = 'SibSp', y= model.predict(final_train), data = final_train, color = 'blue') 
model.llf

In [ ]:
# Parch
sns.scatterplot(x = jitter(final_train.Parch,0), y = jitter(final_train.Survived,0), color = 'orange')
model = smf.logit(formula = 'Survived~Parch', data = final_train).fit()
sns.lineplot(x = 'Parch', y= model.predict(final_train), data = final_train, color = 'blue') 
model.llf

In [ ]:
# Parch squared
sns.scatterplot(x = jitter(final_train.Parch,0), y = jitter(final_train.Survived,0), color = 'orange')
model = smf.logit(formula = 'Survived~Parch+I(Parch**2)', data = final_train).fit()
sns.lineplot(x = 'Parch', y= model.predict(final_train), data = final_train, color = 'blue') 
model.llf

In [ ]:
# age not binned
sns.scatterplot(x = jitter(final_train.Age,0), y = jitter(final_train.Survived,0), color = 'orange')
model = smf.logit(formula = 'Survived~Age', data = final_train).fit()
sns.lineplot(x = 'Age', y= model.predict(final_train), data = final_train, color = 'blue') 
model.llf

In [ ]:
# age binned
sns.scatterplot(x = jitter(final_train.Age,0), y = jitter(final_train.Survived,0), color = 'orange')
model = smf.logit(formula = 'Survived~age_binned', data = final_train).fit()
sns.lineplot(x = 'Age', y= model.predict(final_train), data = final_train, color = 'blue') 
model.llf

In [ ]:
# age squared
sns.scatterplot(x = jitter(final_train.Age,0), y = jitter(final_train.Survived,0), color = 'orange')
model = smf.logit(formula = 'Survived~Age+I(Age**2)', data = final_train).fit()
sns.lineplot(x = 'Age', y= model.predict(final_train), data = final_train, color = 'blue') 
model.llf

In [ ]:
sns.countplot(x = 'age_binned',hue = 'Survived', data = final_train )

In [ ]:
final_train.head()

In [ ]:
X_train1 = X_train.copy()
X_train1['Age_binned'] = final_train['age_binned']
X_train1['agePclass']= final_train.Age * final_train.Pclass
X_train1['ParchSibSp']= final_train.SibSp * final_train.Parch
X_train1['EmbarkedPclass']= final_train.Embarked * final_train.Pclass
X_train1 = X_train1.drop(['Age_binned'], axis = 1)

In [ ]:
X_test1 = X_test.copy()
X_test1['Parch'] = pd.to_numeric(X_test1.Parch)
X_test1['SibSp'] = pd.to_numeric(X_test1.SibSp)
X_test1['Pclass'] = pd.to_numeric(X_test1.Pclass)
X_test1['agePclass']= X_test1.Age * X_test1.Pclass
X_test1['ParchSibSp']= X_test1.SibSp * X_test1.Parch
X_test1['EmbarkedPclass']= X_test1.Embarked * X_test1.Pclass

### Code fitting the final model

Put the code(s) that fit the final model(s) in separate cell(s), i.e., the code with the `.ols()` or `.logit()` functions.

In [ ]:
sklearn_model_1 = LogisticRegression()
sklearn_model_1.fit(X_train1, y_train)

In [ ]:
y_pred_test = sklearn_model_1.predict(X_test1)
print('Accuracy of logistic regression on test set : {:.4f}'.format(sklearn_model_1.score(X_test1, y_test)))

In [ ]:
print(classification_report(y_test, y_pred_test))

### Performance Measurement 

In [ ]:
# confusion matrix
cm = confusion_matrix(y_test, y_pred_test) 
print ("Confusion Matrix : \n", cm) 
  
# accuracy score of the model
print('Test accuracy = ', accuracy_score(y_test, y_pred_test))

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=sklearn_model_1.classes_)
disp.plot()

plt.show()

In [ ]:
metrics.plot_roc_curve(sklearn_model_1, X_test1, y_test)
plt.show()

In [ ]:
ypred = sklearn_model_1.predict(X_train1)
y = y_train
# ypred = logit_model_diabetes.predict(train)
p, r, thresholds = precision_recall_curve(y, ypred)
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.figure(figsize=(8, 8))
    plt.title("Precision and Recall Scores as a function of the decision threshold")
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall")
    plt.ylabel("Score")
    plt.xlabel("Decision Threshold")
    plt.legend(loc='best')
    plt.legend()
plot_precision_recall_vs_threshold(p, r, thresholds)

## Conclusions and Recommendations to stakeholder(s)

# You may or may not have code to put in this section. Delete this section if it is irrelevant.